In [ ]:
from utils.model_utils import load_base_models
from PIL import Image
from torchvision import transforms
import torch
import os

# 加载模型
_, _, seg = load_base_models()

# 图像转换
color_image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 定义图像文件夹路径和输出文件夹路径
input_folder = 'test_images/loukonghair/'
output_folder = 'output_masks/bisenet/'

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 遍历图像文件夹
for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # 读取图像
        color_ref_img = Image.open(os.path.join(input_folder, filename))

        # 图像转换
        color_cond = color_image_transform(color_ref_img).unsqueeze(0).cuda()

        # 获取分割结果
        if color_cond.shape[-1] != 1:
            labels_predict = torch.argmax(seg(color_cond)[0], dim=1).unsqueeze(1).long().detach()
            face_classes = torch.tensor([10], device=labels_predict.device)
            face_mask = torch.isin(labels_predict, face_classes).float().cuda()

            # 将非蒙版部分设为0
            color_cond[0] *= face_mask[0]

            denormalize = transforms.Normalize(mean=[-1, -1, -1], std=[2, 2, 2])
            color_cond_denormalized = denormalize(color_cond[0]).clamp(0, 1)

            # 生成输出文件路径
            output_path = os.path.join(output_folder, filename.replace(".", "_segmented."))
            
            # 保存分割结果
            pil_image = transforms.ToPILImage()(color_cond_denormalized.cpu())
            pil_image.save(output_path)

print("Segmentation and saving complete.")


## 测试IDS

In [ ]:
import cv2
import numpy as np
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

inference = pipeline("face_recognition", model='bubbliiiing/cv_retinafce_recognition', model_revision='v1.0.3')

img1 = 'test_images/ref_img/000.jpg'
img2 = 'test_images/src_img/5.png'
emb1 = inference(dict(user=img1))[OutputKeys.IMG_EMBEDDING]
emb2 = inference(dict(user=img2))[OutputKeys.IMG_EMBEDDING]
sim = np.dot(emb1[0], emb2[0])
print(f'Face cosine similarity={sim:.3f}, img1:{img1}  img2:{img2}')

In [ ]:
import os

# 指定文件夹路径
folder_path = 'test_images/celeba_hq256_100_w'

# 初始化计数器
i = 0

# 遍历文件夹中的文件
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        # 提取文件名和扩展名
        name, ext = os.path.splitext(filename)
        
        # 格式化新的文件名，使用zfill(5)确保数字部分总长度为5，并在前面用0填充
        new_name = f"{i:05d}{ext}"
        
        # 构建旧路径和新路径
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_name)
        
        # 重命名文件
        os.rename(old_path, new_path)
        
        # 计数器递增
        i += 1

print("重命名完成！")


## e4e和原图的余弦相似度均值

In [ ]:
import os
import cv2
import numpy as np
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from tqdm import tqdm  # 导入tqdm

def calculate_cosine_similarity(img_path1, img_path2):
    inference = pipeline("face_recognition", model='bubbliiiing/cv_retinafce_recognition', model_revision='v1.0.3')
    
    emb1 = inference(dict(user=img_path1))[OutputKeys.IMG_EMBEDDING]
    emb2 = inference(dict(user=img_path2))[OutputKeys.IMG_EMBEDDING]
    sim = np.dot(emb1[0], emb2[0])
    
    return sim

def calculate_mean_cosine_similarity(directory1, directory2):
     # 获取目录中的所有文件，并按照数字顺序排序
    files1 = sorted(os.listdir(directory1))
    files2 = sorted(os.listdir(directory2))
    
    # 确保两个目录中的文件数量相同
    # assert len(files1) == len(files2), "目录中的文件数量不一致"
    
    # 存储每对图片的余弦相似度
    similarities = []
    
    # 遍历每一对图片，使用tqdm创建一个进度条
    for file1, file2 in tqdm(zip(files1, files2), total=len(files1), desc="计算相似度"):
        img_path1 = os.path.join(directory1, file1)
        img_path2 = os.path.join(directory2, file2)
        
        # 计算余弦相似度
        similarity = calculate_cosine_similarity(img_path1, img_path2)
        print(similarity)
        similarities.append(similarity)
    
    # 计算均值
    mean_similarity = np.mean(similarities)
    
    return mean_similarity

# 两个目录的路径
path1 = 'test_images/partbarbershop/'
path2 = 'test_images/celeba_hq256_100/'
#path2 = 'test_images/celeba_hq256_100_w/'

# 计算余弦相似度的均值
mean_similarity = calculate_mean_cosine_similarity(path1, path2)

print(f"均值余弦相似度: {mean_similarity:.3f}")


## 计算SSIM和PSNR

In [ ]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import numpy as np

def calculate_metrics(image1_path, image2_path):
    # Read images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)

    # Convert images to grayscale if needed
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM
    ssim_value, _ = ssim(img1_gray, img2_gray, full=True)

    # Calculate PSNR
    psnr_value = psnr(img1, img2)

    return ssim_value, psnr_value

def calculate_mean_metrics(folder1, folder2):
    ssim_values = []
    psnr_values = []

    for filename1, filename2 in tqdm(zip(folder1, folder2), total=len(folder1), desc="计算相似度"):
        if filename1.endswith('.jpg') or filename1.endswith('.png'):
            image1_path = os.path.join(folder1, filename1)
        if filename2.endswith('.jpg') or filename1.endswith('.png'):
            image2_path = os.path.join(folder2, filename)         

        if os.path.exists(image1_path):
            ssim_value, psnr_value = calculate_metrics(image1_path, image2_path)
            ssim_values.append(ssim_value)
            psnr_values.append(psnr_value)

    mean_ssim = np.mean(ssim_values)
    mean_psnr = np.mean(psnr_values)

    return mean_ssim, mean_psnr

if __name__ == "__main__":
    folder1_path = "test_images/partbarbershop/"
    folder2_path = "test_images/celeba_hq256_100/"

    mean_ssim, mean_psnr = calculate_mean_metrics(folder1_path, folder2_path)

    print(f"Mean SSIM: {mean_ssim:.4f}")
    print(f"Mean PSNR: {mean_psnr:.4f}")


In [ ]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import numpy as np

def calculate_metrics(image1_path, image2_path):
    # Read images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)

    # Convert images to grayscale if needed
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM
    ssim_value, _ = ssim(img1_gray, img2_gray, full=True)

    # Calculate PSNR
    psnr_value = psnr(img1, img2)

    return ssim_value, psnr_value

def calculate_mean_metrics(folder1, folder2):
    ssim_values = []
    psnr_values = []

    # Get a list of sorted filenames
    filenames1 = sorted(os.listdir(folder1))
    filenames2 = sorted(os.listdir(folder2))

    for filename1, filename2 in zip(filenames1, filenames2):
        if (filename1.endswith('.jpg') or filename1.endswith('.png')) and (filename2.endswith('.jpg') or filename2.endswith('.png')):
            image1_path = os.path.join(folder1, filename1)
            image2_path = os.path.join(folder2, filename2)

            ssim_value, psnr_value = calculate_metrics(image1_path, image2_path)
            ssim_values.append(ssim_value)
            psnr_values.append(psnr_value)

    mean_ssim = np.mean(ssim_values)
    mean_psnr = np.mean(psnr_values)

    return mean_ssim, mean_psnr

if __name__ == "__main__":
    folder1_path = "output_masks/cross1/"
    folder2_path = "output_masks/cross2/"

    mean_ssim, mean_psnr = calculate_mean_metrics(folder1_path, folder2_path)

    print(f"Mean SSIM: {mean_ssim:.4f}")
    print(f"Mean PSNR: {mean_psnr:.4f}")


## 获得图片除了头发蒙版外相交区域

In [ ]:
from utils.model_utils import load_base_models
from PIL import Image
from torchvision import transforms
import torch
import cv2

# 加载模型
_, _, seg = load_base_models()

path1 = 'test_images/celeba_hq256_100'
path2 = 'test_images/partbarbershop'
files1 = sorted(os.listdir(path1))
files2 = sorted(os.listdir(path2))

# 图像转换
color_image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
i=0
for  img1_path,img2_path in tqdm(zip(files1,files2)):
    img1 = Image.open(f'{path1}/{img1_path}')
    img2 = Image.open(f'{path2}/{img2_path}')
    img1 = color_image_transform(img1).unsqueeze(0).cuda()
    img2 = color_image_transform(img2).unsqueeze(0).cuda()

    if img1.shape[-1] != 1 & img2.shape[-1] != 1:
       
        labels_predict1 = torch.argmax(seg(img1)[0], dim=1).unsqueeze(1).long().detach()
        face_classes1 = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 13],device=labels_predict1.device)
        face_mask1 = torch.isin(labels_predict1, face_classes1).float().cuda()

        labels_predict2 = torch.argmax(seg(img2)[0], dim=1).unsqueeze(1).long().detach()
        face_classes2 = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 13],device=labels_predict2.device)
        face_mask2 = torch.isin(labels_predict2, face_classes2).float().cuda()


        cross_mask = face_mask1*face_mask2
        img1_masked = img1[0]*cross_mask[0]
        img2_masked = img2[0]*cross_mask[0]

        denormalize = transforms.Normalize(mean=[-1, -1, -1], std=[2, 2, 2]) # 撤销之前的标准化（反标准化），使得图像像素变成原来的颜色
        img1_masked = denormalize(img1_masked).clamp(0, 1) # 确保图像中的像素值不超出合法范围。这通常用于处理反标准化后可能出现的数值不精确性或微小的超出范围情况
        img2_masked = denormalize(img2_masked).clamp(0, 1)
        i+=1
        pil_image1 = transforms.ToPILImage()(img1_masked.cpu())
        pil_image1.save(f'output_masks/cross1/{i}.jpg')
        #pil_image1.show()
        pil_image2 = transforms.ToPILImage()(img2_masked.cpu())
        pil_image2.save(f'output_masks/cross2/{i}.jpg')
        #pil_image2.show()

# 获得头发区域，其余区域为0（纯黑）

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
from utils.model_utils import load_base_models
from PIL import Image
from torchvision import transforms
import torch

# 加载模型
_, _, seg = load_base_models()

# 读取图像
color_cond = 'output.png'
color_ref_img = Image.open(f'test_images/ref_img/{color_cond}') # PIL h:1024 w:1024

# 图像转换
color_image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
color_cond = color_image_transform(color_ref_img).unsqueeze(0).cuda() # tensor batch:1 c:3,1024,1024

# 获取分割结果
if color_cond.shape[-1] != 1:
    labels_predict = torch.argmax(seg(color_cond)[0], dim=1).unsqueeze(1).long().detach()
    hair_mask = (labels_predict == 10).float()

    # 将非蒙版部分设为0
    # non_hair_mask = 1 - hair_mask
    color_cond[0] *= hair_mask[0]

    denormalize = transforms.Normalize(mean=[-1, -1, -1], std=[2, 2, 2])
    color_cond_denormalized = denormalize(color_cond[0]).clamp(0, 1)
    color_cond_denormalized *= hair_mask[0]
    # color_cond = color_cond.clamp(0, 1)

# 显示图像
# pil_image = transforms.ToPILImage()(color_cond.cpu().squeeze())
# pil_image.show()
# pil_image.save('output_masks/color/00.jpg')

# 显示图像
pil_image = transforms.ToPILImage()(color_cond_denormalized.cpu())
pil_image.show()
pil_image.save('output_masks/color/118mask.jpg')

# 尝试用SLIC改进颜色选取

In [ ]:
# import the necessary packages
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
 
# load the image and convert it to a floating point data type

image = img_as_float(io.imread("output_masks/color/5mask.jpg"))
 
# loop over the number of segments
for numSegments in (100,):
    # apply SLIC and extract (approximately) the supplied number
    segments = slic(image, n_segments = numSegments, sigma = 2, compactness=0.2)
    # sigma：sigma 控制在计算图像梯度时使用的高斯平滑的标准差。
    # 较小的 sigma 值会导致对图像边缘敏感的梯度计算，从而更容易产生超像素的边缘。
    # 较大的 sigma 值会导致对图像整体结构的平滑，从而在生成超像素时更关注整体颜色分布而不是细节。
    # compactness：
    # compactness 控制了超像素紧凑性的权重。较大的 compactness 会使生成的超像素更加紧凑，即更趋向于规则形状。
    # compactness 参数的值越大，超像素的形状就越接近正方形。较小的值会导致不规则形状的超像素。

    # 设置过滤超参数
    min_nonzero_ratio_threshold = 0.2  
 
    for segment_id in np.unique(segments):
        # Create a mask for the current segment
        mask = (segments == segment_id)

        nonzero_ratio = np.count_nonzero(image[mask]) / np.count_nonzero(mask)

        # 将大部分为黑色的色块过滤掉
        if nonzero_ratio >= min_nonzero_ratio_threshold:
            # Calculate area for the current segment
            area = np.sum(mask)

            # Extract the segment from the original image
            segmented_image = image.copy()
            segmented_image[~mask] = 0  # Set pixels outside the mask to zero
            # Save the segmented image
            plt.imsave(f"output_masks/SLIC/20_10/segment2_{numSegments}_{segment_id}.png", segmented_image)
        

    fig = plt.figure("Superpixels -- %d segments" % (numSegments))
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(mark_boundaries(image, segments))
    plt.axis("off")

    plt.savefig(f"superpixels_{numSegments}.png", bbox_inches='tight', pad_inches=0.0)
 
plt.show()

In [ ]:
from skimage.segmentation import slic, mark_boundaries, find_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import numpy as np

# 加载图像并将其转换为浮点数格式
image = img_as_float(io.imread("output_masks/color/00c.jpg"))

# 循环处理不同分割数
for numSegments in (3, 5, 2):
    # 应用 SLIC 算法并提取给定数量的分割区域
    segments = slic(image, n_segments=numSegments, sigma=5)

    # 查找分割区域之间的边界
    segment_boundaries = find_boundaries(segments, connectivity=1, mode='outer')

    # 遍历每个分割区域并单独保存
    for segment_id in np.unique(segments):
        # 为当前分割区域创建掩码
        mask = (segments == segment_id)

        # 排除属于边界的像素
        mask[segment_boundaries] = False

        # 从原始图像中提取分割区域
        segmented_image = image.copy()
        segmented_image[~mask] = 0  # 将掩码外的像素置为零

        # 保存分割图像
        plt.imsave(f"output_masks/SLIC/segment_{numSegments}_{segment_id}.png", segmented_image)

# 显示图像
plt.show()


# API阿里头发分割

## 把图像抠图获得蒙版但是是RGB格式

In [ ]:
import os
from urllib.request import urlopen
from alibabacloud_imageseg20191230.client import Client
from alibabacloud_imageseg20191230.models import SegmentHairAdvanceRequest
from alibabacloud_tea_openapi.models import Config
from alibabacloud_tea_util.models import RuntimeOptions
from PIL import Image
import numpy as np
import io

os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'] = '***'
os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET'] = '***'

config = Config(
    access_key_id=os.environ.get('ALIBABA_CLOUD_ACCESS_KEY_ID'),
    access_key_secret=os.environ.get('ALIBABA_CLOUD_ACCESS_KEY_SECRET'),
    endpoint='imageseg.cn-shanghai.aliyuncs.com',
    region_id='cn-shanghai'
)

img = open(r'test_images/ref_img/5.png', 'rb')

segment_hair_request = SegmentHairAdvanceRequest()
segment_hair_request.image_urlobject = img
runtime = RuntimeOptions()

try:
    # 初始化Client
    client = Client(config)
    response = client.segment_hair_advance(segment_hair_request, runtime)
    response_data = response.body
    image_url = response_data.data.elements[0].image_url
    height = response_data.data.elements[0].height
    width = response_data.data.elements[0].width
    x = response_data.data.elements[0].x
    y = response_data.data.elements[0].y
    original_width = 1024
    original_height = 1024
    restored_image = Image.new('RGB', (original_width, original_height), color=(0, 0, 0))
    restored_image_array = np.array(restored_image)


    if image_url:
        image_data = urlopen(image_url).read()

        # 获取图片
        image = Image.open(io.BytesIO(image_data))

        # 调整位置大小

        # 指定背景色（可以根据需求选择）
        background_color = (0, 0, 0)  # 这里选择黑色

        # 创建一个黑色背景的RGB图像
        rgba_background = Image.new('RGBA', (original_width,original_height), background_color)

        # 把背景和图像转化为numpy
        image_array = np.array(image,dtype=np.uint8)
        rgba_background_array = np.array(rgba_background,dtype=np.uint8)

        #设置阈值a,Alpha 通道值小于某个阈值 a 时，将 RGB 通道全部设置为零
        a = 220  # 你可以根据需要调整阈值
        # 将 Alpha 通道小于阈值的区域的 RGB 通道设置为零
        alpha_channel = image_array[:, :, 3]
        image_array[alpha_channel < a, :3] = 0
        image_array[alpha_channel >= a, :3] = 255

        # 依据y,x,heigth,width得出图像在背景中的位置
        rgba_background_array[y:y + height, x:x + width] = image_array 

        image_pil = Image.fromarray(rgba_background_array, 'RGBA')

        result_image = Image.alpha_composite(Image.new('RGBA', rgba_background.size, (0, 0, 0, 0)), image_pil)

        result_image = result_image.convert('RGB')
        result_image.save("output_rgb_image.png")

        # 现在你可以将 'image_array' 用作 NumPy 变量
        print('图像形状:', result_image.shape)

        with open('output_image.png', 'wb') as f:
            f.write(image_data)
        print('图像保存到本地文件: output_image.png')

    else:
        print('响应中未找到图像URL。')

    # 获取整体结果
    print(response.body)
except Exception as error:
    # 获取整体报错信息
    print(error)


In [ ]:
import os
from urllib.request import urlopen
from alibabacloud_imageseg20191230.client import Client
from alibabacloud_imageseg20191230.models import SegmentHairAdvanceRequest
from alibabacloud_tea_openapi.models import Config
from alibabacloud_tea_util.models import RuntimeOptions
from PIL import Image
import numpy as np
import io

os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'] = '***'
os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET'] = '***'

config = Config(
access_key_id=os.environ.get('ALIBABA_CLOUD_ACCESS_KEY_ID'),
access_key_secret=os.environ.get('ALIBABA_CLOUD_ACCESS_KEY_SECRET'),
endpoint='imageseg.cn-shanghai.aliyuncs.com',
region_id='cn-shanghai'
)

# 定义图像文件夹路径和输出文件夹路径
input_folder = 'test_images/loukonghair/'
output_folder = 'output_images/'

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 遍历图像文件夹
for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # 读取图像
        img_path = os.path.join(input_folder, filename)
        img = open(img_path, 'rb')

        segment_hair_request = SegmentHairAdvanceRequest()
        segment_hair_request.image_urlobject = img
        runtime = RuntimeOptions()

        # 初始化Client
        client = Client(config)
        response = client.segment_hair_advance(segment_hair_request, runtime)
        response_data = response.body
        image_url = response_data.data.elements[0].image_url
        height = response_data.data.elements[0].height
        width = response_data.data.elements[0].width
        x = response_data.data.elements[0].x
        y = response_data.data.elements[0].y
        original_width = 1024
        original_height = 1024

    if image_url:
        image_data = urlopen(image_url).read()
        with open('output_image.png', 'wb') as f:
            f.write(image_data)
        print('Image saved to local file: output_image.png')

        image = Image.open(io.BytesIO(image_data))

        # 指定背景色（可以根据需求选择）
        background_color = (127, 127, 127)  # 这里选择黑色

        # 创建一个黑色背景的RGB图像
        rgba_background = Image.new('RGBA', (original_width,original_height), background_color)

        # 把背景和图像转化为numpy
        image_array = np.array(image,dtype=np.uint8)
        rgba_background_array = np.array(rgba_background,dtype=np.uint8)

        #设置阈值a,Alpha 通道值小于某个阈值 a 时，将 RGB 通道全部设置为零
        a = 255  # 你可以根据需要调整阈值
        # 将 Alpha 通道小于阈值的区域的 RGB 通道设置为零
        alpha_channel = image_array[:, :, 3]
        image_array[alpha_channel < a, :3] = 127
        #image_array[alpha_channel >= a, :3] = 255

        # 依据y,x,heigth,width得出图像在背景中的位置
        rgba_background_array[y:y + height, x:x + width] = image_array 

        image_pil = Image.fromarray(rgba_background_array, 'RGBA')

        result_image = Image.alpha_composite(Image.new('RGBA', rgba_background.size, (127, 127, 127, 0)), image_pil)

        result_image = np.array(result_image)
        #result_image = result_image[:, :, 0]
        # result_image_path = os.path.join(output_folder, filename.replace(".", "_segmented."))
        # result_image = Image.fromarray(result_image)
        # result_image.save(result_image_path)
        result_image_pil = Image.fromarray(result_image.astype('uint8'))

        # Convert to RGB mode if needed
        result_image_pil = result_image_pil.convert('RGB')
        result_image_path = os.path.join(output_folder, filename.replace(".", "_segmented."))
        result_image_pil.save(result_image_path)

